# Basic buildling blocks

Get to know the basic building blocks of Superlinked.

1. Describe your data using Python classes with the [@schema](https://github.com/superlinked/superlinked-alpha/blob/main/docs/superlinked/framework/common/schema/schema.md) decorator.
2. Describe your vector embeddings with [Spaces](https://github.com/superlinked/superlinked-alpha/blob/main/docs/superlinked/framework/dsl/space/index.md).
3. Combine your embeddings into a queryable [Index](https://github.com/superlinked/superlinked-alpha/blob/main/docs/superlinked/framework/dsl/index/index.m.md).
4. Define your search with dynamic parameters and weights as a [Query](https://github.com/superlinked/superlinked-alpha/blob/main/docs/superlinked/framework/dsl/query/query.md).
5. Load your data using a [Source](https://github.com/superlinked/superlinked-alpha/blob/main/docs/superlinked/framework/dsl/source/index.md).
6. Run your configuration with an [Executor](https://github.com/superlinked/superlinked-alpha/blob/main/docs/superlinked/framework/dsl/executor/in_memory/in_memory_executor.md).

In [ ]:
%pip install superlinked==3.6.3

In [ ]:
from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import String
from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.dsl.space.text_similarity_space import TextSimilaritySpace
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.query.param import Param
from superlinked.framework.dsl.query.query import Query
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)

Create a schema for your data.

In [3]:
@schema
class ParagraphSchema:
    body: String
    id: IdField

Instantiate a new instance of your schema to start the pipeline definition.

In [4]:
paragraph = ParagraphSchema()

Create a space that will run a transformers model on the body of the paragraph.

In [5]:
relevance_space = TextSimilaritySpace(text=paragraph.body, model="all-MiniLM-L6-v2")

Group your space in an index to make it retrievable.

In [6]:
paragraph_index = Index(relevance_space)

Define a query that will search for similar paragraphs in the index. The parameters will be filled later on.

In [7]:
query = (
    Query(paragraph_index)
    .find(paragraph)
    .similar(relevance_space.text, Param("query_text"))
)

Create an in-memory source and executor to try out your configuration

In [8]:
source: InMemorySource = InMemorySource(paragraph)
executor = InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

Insert some example data.

In [9]:
source.put([{"id": "happy_dog", "body": "That is a happy dog"}])
source.put([{"id": "happy_person", "body": "That is a very happy person"}])
source.put([{"id": "sunny_day", "body": "Today is a sunny day"}])

Query your data.

In [10]:
result = app.query(query, query_text="This is a happy person")

print(result)

#1 id:happy_person, object:{'body': 'That is a very happy person', 'id': 'happy_person'}
#2 id:happy_dog, object:{'body': 'That is a happy dog', 'id': 'happy_dog'}
#3 id:sunny_day, object:{'body': 'Today is a sunny day', 'id': 'sunny_day'}


Check how a different query can produce different results.

In [11]:
result = app.query(query, query_text="This is a happy dog")

print(result)

#1 id:happy_dog, object:{'body': 'That is a happy dog', 'id': 'happy_dog'}
#2 id:happy_person, object:{'body': 'That is a very happy person', 'id': 'happy_person'}
#3 id:sunny_day, object:{'body': 'Today is a sunny day', 'id': 'sunny_day'}
